In [1]:
# Modelagem dos dados
import numpy as np
import pandas as pd


In [2]:
# Conecta o notebook ao Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Importa base de dados do pib em dolares correntes dos países
gdp = pd.read_excel('/content/drive/MyDrive/Pesquisa - Talles/Padrões de normas vigentes no comércio internacional: caracterização e análise do efeito da norma Globalgap sobre as exportações de frutas no Brasil, Chile e Argentina./Dados/PIB Nominal dos parceiros comerciais/PIB_paises/world_bak_gdp_usd.xlsx')


---

In [8]:
# Importa base de dados do pib em dolares correntes dos países
parceiros = pd.read_excel('/content/drive/MyDrive/Pesquisa - Talles/Padrões de normas vigentes no comércio internacional: caracterização e análise do efeito da norma Globalgap sobre as exportações de frutas no Brasil, Chile e Argentina./Dados/PIB Nominal dos parceiros comerciais/Base_exp_parceiroscomerciais/base_paises_parceiroscomerciais.xlsx', sheet_name=0)
parceiros.head()


,ano,ncm,fruta,uf,pais,fob,kg_liq
0,2018,8044000,Abacates frescos ou secos,Alagoas,Grécia,14,5
1,2018,8055000,"Limões (Citrus limon, Citrus limonum) e limas ...",Alagoas,Grécia,140,63
2,2018,8081000,Maçãs frescas,Alagoas,Bahamas,475,161
3,2018,8081000,Maçãs frescas,Alagoas,França,148,36
4,2018,8081000,Maçãs frescas,Alagoas,Grécia,449,170


In [9]:
# Reorganiza os dados!
df = parceiros.pivot_table(index=['ano', 'uf', 'fruta'], columns='pais', values='fob')

In [10]:
# Coluna com o total das exportações por fruta e por estado
df['exp_USD'] = df.sum(axis=1)


In [11]:
# Mover o somatório das exportaações para o início do DataFrame
# Remove a coluna a ser movida
coluna_a_mover = df.pop('exp_USD')

# Insere a coluna na primeira posição
df.insert(0, 'exp_USD', coluna_a_mover)


In [12]:
# Preenche os valores nulos com zeros!
df = df.fillna(0)


In [14]:
# Calcula o percentual para cada país (exceto a coluna 'Total')
df_percentual = df.drop(columns=['exp_USD']).div(df['exp_USD'], axis=0)

# Adiciona as colunas de percentual ao DataFrame original
df = pd.concat([df, df_percentual.add_suffix('_perc')], axis=1)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 702 entries, (2018, 'Alagoas', 'Abacates frescos ou secos') to (2022, 'São Paulo', 'Uvas frescas')
Columns: 287 entries, exp_USD to Índia_perc
dtypes: float64(287)
memory usage: 1.5+ MB


In [16]:
df.to_excel('dados.xlsx')

---

In [ ]:
gdp.rename({'Unnamed: 3' : 'pais'}, axis=1, inplace=True)

In [ ]:
gdp = gdp.dropna()

In [ ]:
# Dicionário
# Substitui os nomes de gdp['paises'] para padronizar com parceiros['pais']

substituicoes = {
    'Bielorrússia': 'Belarus',
    'Bósnia e Herzegovina':'Bósnia-Herzegovina',
    'Ilhas Cayman': 'Cayman, Ilhas',
    'Cingapura': 'Singapura',
    'Congo, Rep.': 'Congo',
    'Coreia, Dem. Representante do Povo': 'Coreia do Norte',
    'Coreia, Rep.': 'Coreia do Sul',
    'Ilha de Man': 'Ilha de Man',
    'ilhas Faroe': 'Faroe, Ilhas',
    'Ilhas Marshall': 'Marshall, Ilhas',
    'Ilhas Turcas e Caicos': 'Turcas e Caicos, Ilhas',
    'Ilhas Virgens (EUA)': 'Virgens, Ilhas (Americanas)',
    'Irã, Rep. Islâmica.': 'Irã',
    'RAE de Hong Kong, China': 'Hong Kong',

}


In [ ]:
# Aplica as substituições
gdp['pais'] = gdp['pais'].replace(substituicoes)


In [ ]:
# Une a base inicial com a base do PIB dos países
base = gdp.merge(parceiros, on='pais', how='inner')


In [ ]:
# Exclui colunas desnecessárias
base = base.drop(columns=['Series Name', 'Series Code', 'Country Name', 'Country Code',
                          'ncm', 'kg_liq'], axis=1)


In [ ]:
# Transforma as colunas de anos em linhas (melt)
gdp_melted = gdp.melt(id_vars='pais', value_vars=['2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]', '2021 [YR2021]', '2022 [YR2022]'], var_name='ano', value_name='PIB')


In [ ]:
# Substitui os valores na coluna 'ano'
valores = {
    '2018 [YR2018]' : 2018,
    '2019 [YR2019]' : 2019,
    '2020 [YR2020]' : 2020,
    '2021 [YR2021]' : 2021,
    '2022 [YR2022]' : 2022

}

In [ ]:
# Substitui valores
gdp_melted['ano'] = gdp_melted['ano'].replace(valores)


In [ ]:
# Unindo os DataFrames pelo nome dos países e o ano
base = parceiros.merge(gdp_melted, on=['pais', 'ano'], how='inner')


In [ ]:
# Remove as strings '..'
base['PIB'] = base['PIB'].replace('..', np.nan)

# Converte valores para float
base['PIB'] = base['PIB'].astype(float)


In [ ]:
# Organiza os dados
df2 = base.pivot_table(index=['ano', 'uf', 'fruta'], columns='pais', values=['fob', 'PIB'])


---

## Correção de alguns dados

In [ ]:
dados = pd.read_excel('/content/drive/MyDrive/Pesquisa - Talles/Padrões de normas vigentes no comércio internacional: caracterização e análise do efeito da norma Globalgap sobre as exportações de frutas no Brasil, Chile e Argentina./Dados/PIB Nominal dos parceiros comerciais/base_ponderacao.xlsx', sheet_name=1)

In [ ]:
dados = dados.fillna(0)

In [ ]:
dados.to_excel('dados_zeros.xlsx', index=False)

---

In [3]:
df = pd.read_excel('/content/drive/MyDrive/Pesquisa - Talles/Padrões de normas vigentes no comércio internacional: caracterização e análise do efeito da norma Globalgap sobre as exportações de frutas no Brasil, Chile e Argentina./Dados/PIB Nominal dos parceiros comerciais/base_ponderacao.xlsx')


In [5]:
df.shape

(702, 576)